In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\lokes\OneDrive\Desktop\NLP Krish\Fake news\train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
###Drop Nan Values
df=df.dropna()


In [7]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [8]:
## Get the Dependent features
y=df['label']

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.11.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
### Vocabulary size
voc_size=5000

# Onehot Representation

In [15]:
messages=X.copy()

In [16]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lokes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

# Initialize the Porter Stemmer
ps = PorterStemmer()

# Initialize the list to store the preprocessed text
corpus = []

# Loop through each row in the 'title' column
for i, row in messages.iterrows():
    # Remove non-alphabetic characters and convert to lowercase
    review = re.sub('[^a-zA-Z]', ' ', str(row['title'])).lower()
    
    # Tokenize the text (split it into words)
    review = review.split()
    
    # Apply stemming and remove stopwords
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    
    # Join the words back into a sentence
    review = ' '.join(review)
    
    # Append the preprocessed text to the corpus
    corpus.append(review)



In [21]:
# Check column names
print(messages.columns)

# Check the number of rows
print(len(messages))

Index(['id', 'title', 'author', 'text'], dtype='object')
18285


In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [25]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [26]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2579, 2764, 1047, 1442, 170, 3527, 4907, 211, 2463, 485],
 [633, 4072, 743, 4966, 3444, 2111, 4683],
 [1170, 2041, 3494, 3413],
 [3402, 2539, 3309, 2352, 109, 4095],
 [4291, 3444, 133, 1850, 3793, 3711, 3444, 2688, 2147, 3465],
 [933,
  2274,
  2251,
  3797,
  1250,
  2399,
  202,
  3471,
  4371,
  1819,
  1299,
  215,
  3725,
  3771,
  4683],
 [978, 1941, 1462, 2878, 1580, 4230, 1000, 3492, 4542, 4786, 442],
 [426, 206, 4750, 4964, 4989, 466, 2399, 1344, 4542, 4786, 442],
 [3054, 4004, 4531, 2123, 1168, 264, 3272, 2362, 2399, 1618],
 [4278, 960, 4082, 896, 3885, 3720, 3082, 2036],
 [2561, 2471, 3575, 2395, 2916, 1531, 2350, 1963, 4918, 1787, 1973],
 [2352, 3114, 170, 264, 2399, 4989],
 [2487, 903, 208, 4621, 160, 131, 938, 2221, 4309],
 [369, 1262, 1391, 2035, 1694, 1361, 807, 4542, 4786, 442],
 [2112, 193, 3185, 2379, 4299, 4542, 4786, 442],
 [1078, 4013, 1505, 2913, 3773, 789, 2210, 2852, 4237, 4881],
 [213, 3958, 4072],
 [2983, 1345, 520, 2467, 2399, 324, 4140, 4683],
 [4609, 118

In [27]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [28]:
onehot_repr[1]

[633, 4072, 743, 4966, 3444, 2111, 4683]

# Embedding Representation

In [29]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2579 2764 1047 ...    0    0    0]
 [ 633 4072  743 ...    0    0    0]
 [1170 2041 3494 ...    0    0    0]
 ...
 [ 152  350   41 ...    0    0    0]
 [4942 4989 4414 ...    0    0    0]
 [1979 2378 3019 ...    0    0    0]]


In [30]:
embedded_docs[1]

array([ 633, 4072,  743, 4966, 3444, 2111, 4683,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [31]:
embedded_docs[0]

array([2579, 2764, 1047, 1442,  170, 3527, 4907,  211, 2463,  485,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [32]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
len(embedded_docs),y.shape

(18285, (18285,))

In [34]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [35]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

# Model Training

In [37]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 13s 43ms/step - loss: 0.3154 - accuracy: 0.8482 - val_loss: 0.2054 - val_accuracy: 0.9152
Epoch 2/10
192/192 [==============================] - 5s 26ms/step - loss: 0.1325 - accuracy: 0.9487 - val_loss: 0.2030 - val_accuracy: 0.9196
Epoch 3/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0809 - accuracy: 0.9736 - val_loss: 0.2272 - val_accuracy: 0.9188
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0522 - accuracy: 0.9820 - val_loss: 0.3216 - val_accuracy: 0.9069
Epoch 5/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0375 - accuracy: 0.9866 - val_loss: 0.3336 - val_accuracy: 0.9127
Epoch 6/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0268 - accuracy: 0.9918 - val_loss: 0.3553 - val_accuracy: 0.9142
Epoch 7/10
192/192 [==============================] - 5s 24ms/step - loss: 0.0182 - accuracy: 0.9944 - val_loss: 0.4158 - val_accuracy: 0.907

# Performance Metrics And Accuracy

In [38]:
y_pred=model.predict(X_test)

189/189 [==============================] - 2s 7ms/step


In [39]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [40]:
from sklearn.metrics import confusion_matrix

In [41]:
confusion_matrix(y_test,y_pred)

array([[3126,  293],
       [ 246, 2370]], dtype=int64)

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9106876553438277

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

